# Sample background 

All guano was collected by Dave Yates and associates at Biodiversity Research Institute over the summer of 2016 at various locations throughout the eastern United States. The following sequence data was generated either from DNA extracted following guano extraction standards developed in the Foster Lab (described elsewhere). Arthropod COI fragments were produced using new JOH primers, then 250 PE sequencing on an Illumina HiSeq2500 generated our reads.  A single lane was used in generating these data, though two rounds of sequencing were used in generating results. The initial run produced suitable quantities of reads for three of nine samples, thus a second round of sequencing was performed. The following results reflect only that second round of sequencing, as indicated in the run statistics shown [here] [link1]. 

[link1]:http://cobb.unh.edu/170125_DevonP2L2_A_DemuxStats.html

# Pipeline Overview

We use the tools developed by Jon Palmer in the [**amptk**] [link1] program which will trim, filter, cluster, and assign taxonomy to reads. The core clustering process employs an algorithm called DADA2 (see [paper] [link3] and [github page] [link4]) which has an added benefit of error correcting reads (even singletons), though each identified cluster is then reclustered at a 97% similarity (as done conventionally in many OTU-clustering approaches) for downstream community analyses. Taxonomic assignment is completed in part using the curated Barcode of Life Database [(BOLD)] [link7], while also leveraging Robert Edgar's [SINTAX] [link5] and [UTAX] [link6] algorithms to provide additional taxonomic information for reads which are not matched in BOLD.

[link1]:https://github.com/nextgenusfs/amptk
[link3]:http://www.nature.com/articles/nmeth.3869.epdf?author_access_token=hfTtC2mxuI5t44WUhsz05NRgN0jAjWel9jnR3ZoTv0N5gu3rLNk61gF4j2hXPcagLe964qdfd3GRw8OwyZxfEgsul8lwR1lEWykR3lWF30Dl_bZWMvTPwOdrwuiBUeYa
[link4]:https://github.com/benjjneb/dada2
[link5]:http://www.biorxiv.org/content/early/2016/09/09/074161
[link6]:http://www.drive5.com/usearch/manual/utax_algo.html
[link7]:http://v4.boldsystems.org/

## Other information

Installation requirements are found on the [amptk] [link1] site. This project was completed with the following dependencies:

- amptk v0.8.5 
- vsearch v2.3.4
- usearch v9.2.64 (linked as "usearch9", not "usearch")  

In addition to the basic tools we're also going to need to have installed a few other items:  
- a fasta file of the mock community (used in the sequence index bleed filtering step)
    - we are using the IM3 mock community provided by Michelle Jusino
- the COI index (either default from 'amptk' or a curated one of your own)

To install the amptk default COI database:

[link1]:https://github.com/nextgenusfs/amptk/blob/master/docs/ubuntu_install.md

In [ ]:
amptk install COI

# part 1 - data migration and cleaning

Initial steps involve moving, renaming, and filtering data as follows.

First, move all relevant directories from Cobb to Pinky servers with rsync. For each project, there will likely be three sets of samples to migrate: 
- the samples
- the negative controls
- one or more mock community members  

This will occur for every lane for every project. In instances where there are multiple lanes in which samples were split across, each lane's data will be treated with separate mock community samples so that index bleed is lane-specific. Data for an entire project is only merged after OTUs have been subsequently determined; this is done in R.  

## generic migration

In [ ]:
rsync -avzr foster@cobb.unh.edu:/path2data/Sample_*/*.gz /copy2here/fqRaw/.

## generic renaming

Rename the files to remove the unnecessary underscore. As there are several different prefixes to deal with, we'll run this command as many times as needed for each unique prefix type (while implementing wildcards when possible).

In [ ]:
#for true samples
rename 's/{name}_/{name}\-/' *

#for any contaminant sample
rename 's/{name}-contaminated_/{name}une-contaminated/' *

#for negative controls
rename 's/neg{name}_/negu{name}\-/' *

#for mock commmunity
rename 's/mock_IM3_/mockIM3\-/' *

Once everything is properly named, you can then proceed. If you get errors in downstream processes circle back and see if you've kept any weird underscores or other characters in the file names. See the [amptk homepage] [link1] if you're unclear about how the sequence data is expected to be named.

[link1]:https://github.com/nextgenusfs/amptk

## One final pre-processing note

If you have any files that contain zero bytes of data (literally no indices were detected during the Illumina run), you need to delete those .fq files ahead of time before trimming with the first step in the 'amptk' pipeline (it crashes when it tries to parse data that doesn't exist!). To delete files with zero bytes of data, enter the following command (within the directory containing all the raw fastq files):

In [ ]:
 find . -name "*.fastq" -size -1c -delete

# Read processing

This process can take a considerable amount of time; increase the number of cpus when possible is advised. For example, with 8 cpus on about 17 Gb of data, it took ~1 hour to process.  

Note the suffix "L1" in the **-o** flag. This is done in this particular circumstance because this project contains data across two lanes. If there were multiple Illumina runs, these would be further specified by project number (ex. P1L1, P3L2, etc.).

In [ ]:
#run in parent directory of {path.to}/fqRaw
amptk illumina \
-i /leo/devon/projects/guano/bri2016/p2_data/fqRaw \
-o briall \
--rescue_forward on \
--require_primer off \
--min_len 160 \
--full_length \
--cpus 20 \
--read_length 250 \
-f GGTCAACAAATCATAAAGATATTGG \
-r GGWACTAATCAATTTCCAAATCC

Output from this script will contain several new files:  

1. In the directory in whch the script was executed:
    - (output_name).mergedPE.log containing information about PE merging. Note that information in this single file includes the summary of all individual merged pairs.
    - (output_name)-filenames.txt containing information about files used in this pipeline (such as index-pair combinations)
    - (output_name).demux.fq containing a concatenated file of all trimmed and PE merged sequences of all samples listed in the '-filenames.txt' file above __(this one is to be used in the subsequent OTU clustering step)__

2. In the (output_name) directory named in the 'ufits illumina ...' argument provided above:  
    - (output_name).ufits-process.log containing information about the demultiplexing and read trimming processes
    - a single sample_name.fq file which is the PE merged file from each sample_name...R1/R2 pair of raw fastq inputs
    - a single sample_name.demux.fq file for each PE merged sample_name.fq file having been trimmed as defined  

The file labeled **'{-o name}.ufits-demux.log'** will also contain a summary of information regarding the total number of reads processed as well as the number of reads processed per sample (at the very bottom of the file). This is critical in evaluating which reads to keep, and the names of those files are essential in the next step for keeping/dropping samples.

## Dropping/keeping certain samples

You may want to exclude certain samples which contain too few reads. These can be discarded at this point before moving forward in the OTU clustering part. Jon Palmer recommends shooting for samples with about 50,000 reads, though as low as 10,000 samples are fine. It also depends on the distribution of the reads across all samples; if you have 20 samples with >100,000 reads each, and 20 samples with ~10,000 you might not want to keep anything less than 10,000 reads. However, if you have 20 samples with ~20,000 reads each and another 10 samples with 5,000 reads, and another 10 samples with ~100 reads, then you might want to keep those with 5,000 (so going lower than the 10,000 reads threshold).

There's an equivalent set of commands which you can specify to keep or remove samples; if it's faster to enter just a few samples to drop, us the '**amptk remove**' command, if it's faster to just specify the few samples you want to keep, use the '**amptk select**' command.  

In this specific run there were a remarkably high number of samples to keep and just a few to drop. Notably, a single negative control appeared to contain a sample of guano. This isn't what minor contamination looks like - that's way too many reads. What's likely happening is that a piece of guano was placed in that well and not identified prior to extracting the DNA; alternatively it could be a situation where we deliberately put guano in a well and mislabeled it. We can keep it in and mark it as a contaminant, and in our community analysis downstream should look and see if it resembles neighboring wells (though this is also problematic because neighboring wells likely contain samples from similar place/times, so you won't know if it's contamination or just replication of information). Here are what the top and bottom of our read depths looked like - note how all the other negative contorls contain 100-fold less reads:

In [ ]:
Sample:  Count
bri16-0135:  296763
bri16-0420:  295297
bri16-0429:  282819
...
negbri-41C03:  149683
bri16-0107:  146466
mockIM3:  107175
...
negbri-41B02:  1243
bri16-0731:  1184
negbri-41F06:  710
...
bri16-0099:  327

We're going to drop any sample with less than 10,000 reads. Could have kept one or two additional samples with slightly less (ex. bri16-0735 had 9,136 reads and bri16-0748 had 7,024 reads, etc.); the next negative control contained only 1731 reads making the 10,000 read cut off somewhat informed. To drop those samples, make a list of the file names to drop in a text file then run this command: 

In [ ]:
amptk remove \
-i briall.demux.fq \
-f samples2drop.txt \
-o bridropd.demux.fq
    #kept about 99.5% of all reads
    #kept 58 of 74 samples

To double check that new .fq file contains just what you want, run the following command to double check you didn't miss anything.

We're good to start with the next part - clustering OTUs.

## part 2 - OTU clustering

See Jon Palmer's [notes] [link1] about DADA2 if you're curious about what's required to get to this point.  

We used to have to clean up reads before passing into DADA2 using the vsearch program, but now the code in amptk deals with this problem just fine.
[link1]:https://github.com/nextgenusfs/ufits

This will produce two files you want to use in the subsequent 'ufits filter' command below:
   - **dada2_output.cluster.otus.fa** (use this fasta file in the next command)
   - **dada2_output.cluster.otu_table.txt** (use this fasta file in the next command)

It also produces two very similar looking files that represent the inferred sequences which were further clustered at 97% identity (so we can make some sort of sense of it when assigning taxonomy) - these aren't necessary for downstream analyses as of now:
   - **dada2_output.iSeqs.fa**
   - **dada2_output.otu_table.txt**

You can see exactly which of those iSeq files ended up clustering together into a single OTU with this file:
   - **iSeqs 2 OTUs: dada2_output.iSeqs2clusters.txt**

There are also a few log files generated:
   - **dada2_output.dada2.Rscript.log**  (to ensure the DADA2 program was run successfully)
   - **dada2_output.ufits-dada2.log**  (tracks the overall processing of this wrapper 'ufits dada2' script)


Next up is to filter the OTU table with the fasta file listed above:

# Part 3: filtering OTU table

It's advocated by Jon Palmer to use an index-bleed filter command using the '-b mock' flag, specifying the use of a mock community to calculate the index-bleed percentage. The idea behind this filtering step is to identify the number of instances in which a read mapped to an OTU which is *not supposed to be in the mock community* is found in the mock community. The percentage represents the overall number of reads that map to mock OTUs (ie. that are supposed to be in there) relative to the number of reads from OTUs that aren't. The alternative approach is to just trim down reads by some defined (yet arbitrary) percent across all samples, given some other examples in other data sets. I don't like that because I've found that every data set is unique, so having a mock community to judge this by is better.  

We're going to use our mock community to calculate index bleed. We're also going to (potentially) incorporate a subtraction value in which *if* the scenario occurs such that an unwanted OTU remains in the mock community following the application of the index-bleed filter, we can detect how many reads that maximum value should be and subtract from there to ensure that **zero** normalized reads remain in the highest 'bleeding' OTU in the mock sample. This value is subtracted from *all* reads from *all* samples per OTU, not just the mock community, so it ultimately drops a lot of OTUs with low read thresholds.  

There are intermediate files which are very useful in determing exactly how many read of which OTU are creeping into your mock community (that are unwanted OTUs), so you'll notice we're passing a **"--debug"** flag which is used to generate these files; without that flag, you'll miss the normalized read counts used in calculating the index bleed filter as well as the subsequent "--subtract" value.

You might want to play with the **--subtract** threshold and examine how many overall OTUs are retained. In the two examples below, we're going to test exactly what filtering parameters should be applied.  

- Our first filtering step uses several flags: 
    - the index-bleed flag **-b** specifies which sample is the mock community and goes with the **--mc** flag to specify the path with which you can find the associated fasta file with that mock community
    - the **calculate in** flag specifies that we're only going to calculate index bleed into the mock (not back out of the mock); this is done because we are using a biological mock community and there is a chance that our true samples also contain the same species (OTUs really); if left to the default (**calculate all**) *and we had samples that contained these mock OTUs too* then our index bleed rate would be super high, which we don't want
    - the **--debug** flag provides the intermediate files needed to determine which OTUs and how many reads are finding their way into the mock community sample
    - the **--subtract auto** flag will calculate what the total number of normalized reads are in OTUs unwated in the mock community and then subtract that value from all reads. When applied this results a completely clean mock, but drops a lot of samples

First example allows for an automatic detection and application of index-bleed and OTU subtraction (if necessary):

In [ ]:
amptk filter \
-i bri.cluster.otu_table.txt \
-f bri.cluster.otus.fa \
-b mockIM3 \
--keep_mock \
--calculate in \
--mc /leo/devon/projects/guano/mock-fa/CFMR_insect_mock3.fa \
--debug \
--subtract auto \
-o testfilt

The resulting output will show you that:
- there was an index bleed of 1.4% applied (that's pretty high!), and 
- the auto subtract filter was used at a level of 1277.  

In other words, for *Lane 1 data*, after filtering down all reads by 1.4% on a per-OTU basis, all reads were then subtracted form each sample's OTU by an additional 1250 reads. If you used those default parameters it would result in a dramatic reduction in the resulting OTUs to keep. For instance in this filtering regime we go from **831 OTUs** down to **170 OTUs** - that's a lot of OTUs to throw away. The question is whether or not that **--auto subtract** filter needs to be quite as high. We're going to figure that out in a minute.  

To investigate what's going on in both data sets we're going check out the **testfilt.normalized.num.txt** file and compare it with the **testfilt.final.txt** file (one for each lane). We need to figure out which OTUs in the mock community have the greater number of OTUs that are *unwanted* - that is, OTUs which should not be in the mock community. We're going to clean up the output of the needed files just a bit so that our search basic linux tools work properly (this is because the space in the first line of the files shifts the first row into one more field than all the rows below).

Run the following commands:

In [ ]:
sed -i 's/#OTU ID/OTUid/' testfilt.final.txt
sed -i 's/#OTU ID/OTUid/' testfilt.normalized.num.txt

You don't know two things at this point: 
- how many fields are there?
- which OTUs contain the most reads among OTUs that **don't** belong?  

To answer the *number of fields question*, substitute **file.name.txt** with whatever file you want:

In [ ]:
awk -F '\t' '{print NF; exit}' file.name.txt

For example:

In [ ]:
awk -F '\t' '{print NF; exit}' testfilt.final.txt

In [ ]:
returns the value

In [ ]:
# 60

So there are 60 fields; that's important when you want to next figure out which columns to sort. With our current data set the second to last field is the mock community (59), and the first field is the list of OTUs in each sample. So we're going to just sort through this big array of a text file and just print out the three fields we want.   

The following command should give you a sense of how many reads may need to be subtracted from a given OTU that's unwanted. In the first command you can view the OTU in question, the mock sample, and another real sample in terms of how many reads are in each OTU.

In [ ]:
#for the .normalized.num.txt file:
cut testfilt.normalized.num.txt -f 1,59 | sort -k2,2n | awk '$2 != "0.0" {print $0}' 

#for the .final.txt file:
cut testfilt.final.txt -f 1,59 | sort -k2,2n | awk '$2 != "0" {print $0}'

Notice how there is a single OTU that shouldn't be in there that's in abundance (**OTU131**) while there are a handful of other OTUs that shoudn't be there but contain only a fraction of reads? I think after looking at multiple sets of samples all sequenced on this Project 2 run it appears that some fly DNA is present in one of the reagents at some very low level (probably in the PCR mix, but possibly anywhere). You can search for this OTU using the second command below. Because this project set includes only bat DNA we know we aren't going to find that fly in it so we can drop this OTU all together.

In [ ]:
grep "\\bOTU131\\b" testfilt.normalized.num.txt

Which will result in the following output (highlighted text are the mock community value):  
OTU131&nbsp;&nbsp;  0.0&nbsp;&nbsp;  3.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  24.0&nbsp;&nbsp;  11.0&nbsp;&nbsp;  8.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  1.0&nbsp;&nbsp;  43.0&nbsp;&nbsp;  5.0&nbsp;&nbsp;  1.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  2.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  3.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  4.0&nbsp;&nbsp;  25.0&nbsp;&nbsp;  1.0&nbsp;&nbsp;  0.0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  3.0&nbsp;&nbsp;  49.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  2.0&nbsp;&nbsp;  1.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  1.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  5.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  3.0&nbsp;&nbsp;  2.0&nbsp;&nbsp;  761.0&nbsp;&nbsp;  33.0&nbsp;&nbsp;  447.0&nbsp;&nbsp;  19.0&nbsp;&nbsp;  57.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  1.0&nbsp;&nbsp;  2.0&nbsp;&nbsp;  46.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  6.0&nbsp;&nbsp;  1.0&nbsp;&nbsp;  340.0&nbsp;&nbsp;  0.0&nbsp;&nbsp;  1.0&nbsp;&nbsp;  **1277.0&nbsp;&nbsp;**  1.0

Interestingly, the mock community contains the highest read count of any sample. Though the there are a handful of other samples with very high read totals, most samples contain just traces of this OTU. This could be an indication of index-bleed, as chimeric sequences would typically how low read abundance in many samples (the opposite of what we see above). The problem is that you'd need another sample with much higher number of reads than our mock to make index bleed work; **keep in mind that other samples were run in this lane with these samples**... and it's likely that index bleed is happening from *non-BRI* samples. 

It'll be useful to identify (as best as possible) the taxonomic information of these potential contaminant OTUs in question. They may be chimeric or index-bleed and this is another way to identify what's going on. It could also be a species that you are sure shouldn't be in there at all, which you can then consider just outright contamination in a reagent somewhere in your prep work. You can do this by looking into the fasta file for each of those OTUs and then manually BLASTing them. For example:

In [ ]:
grep "\\bOTU131\\b" bri.cluster.otus.fa -A 3

This produces the sequence needed; manually BLASTing these specific OTUs in NCBI's nr database show the same result: it's a common housefly. It's unexpected in this dataset because we wouldn't imagine that bats are dining in huge numbers on North American house flies. Interestingly, the other lane in this sequencing run (these data were from Lane2, while another pooled sample was run on the same flow cell in Lane 1) contained a dataset that included some bird guano; in those samples the housefly was in abundance and therefore expected to find its way to bleeding into the mock community, however, it's unlikely that this would affect a different lane entirely. This leads me to the following conclusions:  
- it's probably not a chimerica OTU
- it's possibly the result of index bleed given the huge number of reads present in a handful of bird guano samples but I don't know how you'd end up with multiple-lane bleed... so... 
- It's most likely that it's the result of a contaminant in a reagent (could be at any of the levels of DNA extraction, PCR amplification, or PicoGreen quantification).

Given that information, it'll be easiest to just drop that OTU in question and then refilter. We're going to want to keep the **--subtract auto** feature on for now for both data sets because there were additional low-level reads popping up in their respective mock community samples.

*AMPtk* has a feature to do just that for weird scenarios like this!

In [ ]:
amptk drop \
-i bri.cluster.otus.fa \
-r bridropd.demux.fq \
-l OTU131 \
-o bri

The above command will result in a single OTU being dropped from each .fq file and generate two output files:  
- **bri.cleaned.otu_table.txt** 
- **bri.cleaned.otus.fa**  

Next up is to run another filtering step to ensure that the OTU in question has been removed, and see how the removal of that OTU influences what the **--subtract** value is set to.

In [ ]:
amptk filter \
-i bri.cleaned.otu_table.txt \
-f bri.cleaned.otus.fa \
-b IM3 \
--keep_mock \
--calculate in \
--mc /leo/devon/projects/guano/mock-fa/CFMR_insect_mock3.fa \
--subtract auto \
--debug \
-o lasttest

Big difference. We find that dropping that one OTU results in the lowering of the **--subtract auto** value from the very high range to a more modest number of reads per sample: from **1250** to **47** reads.

While there are still a few additional samples bleeding into the mock community these all occur at a very low abundnace. We can therefore apply a final filtering step where we keep only true samples (we drop the mock community and it's uniquely associated OTUs), and filter down with a more modest subraction filter.

In [ ]:
amptk filter \
-i bri.cleaned.otu_table.txt \
-f bri.cleaned.otus.fa \
-b mockIM3 \
--calculate in \
--mc /leo/devon/projects/guano/mock-fa/CFMR_insect_mock3.fa \
--subtract auto \
--delimiter tsv \
-o bri

So what do we see? By dropping just that one contaminant OTU from each data set we greatly reduce the value used in the **--subtract auto** option. We go from a subract value of **1277** to just **47**; this results in retaining **432** OTUs instead of the **170** that were left when we kept that one contaminant OTU. Big difference!

Notably, 11 of those OTUs are specifically associated with the mock community that is now removed. When you look at the abundance of reads per OTU it's pretty clear that many of these dropped OTUs are very rare (are present only in one or a few of the samples and absent in others), yet often when we see OTUs containing thousdands of reads they tend to be present in many of the samples. To be conservative it's certainly justifiable to cut at the rate with which we elimnate index bleed from our mock community and that's what we'll do here.  

The arguments above will produce the following files:  
- **nau.filtered.otus.fa** is the final filtered fasta file (say that five times fast)
- **nau.final.binary.txt** is the presence/absence OTU table after filtering
- **nau.stats.txt** is the number of OTUs in each sample before and after filtering
- **nau.final.txt** is the OTU table with normalized read counts (noramlized to the number of reads in each sample)
- **nau.amptk-filter.log**  

The next step is to assign taxonomy by using the **nau.filtered.otus.fa** and **nau.final.binary.txt** files. 

## Part 4: Assigning Taxonomy to OTUs

Make sure to have already downloaded the necessary database (COI). If you haven't, just type:

In [ ]:
amptk install -i COI

Note that the classification is performed using three potential programs: USEARCH using our database acquired through BOLD; UTAX (trained through BOLD); and SINTAX.

In [ ]:
amptk taxonomy \
-i bri.final.binary.txt \
-f bri.filtered.otus.fa \
-d COI \
-o bri

## What do we find?

You're going to want to transfer the output file "**dada2_r97m2.otu_table.taxonomy.txt**" to your desktop and use the  R script **'ufits_2016_OTUtable_analysis.R'** for generating preliminary lists of taxa present in these samples.